## 1. Explode 

In [147]:
import pandas as pd
from pathlib import Path

org_data_path = Path.cwd().parent / "data" / "heirarchy.csv" 

pd.set_option("display.max_columns", 85)
pd.set_option("display.max_rows", None)

In [148]:
df = pd.DataFrame({'A': [[1, 2, 3], [], [3, 4]], 'B': 1})
df

,A,B
0,"[1, 2, 3]",1
1,[],1
2,"[3, 4]",1


In [149]:
df = df.explode('A')

In [150]:
df.reset_index()

,index,A,B
0,0,1,1
1,0,2,1
2,0,3,1
3,1,NaN,1
4,2,3,1
5,2,4,1


In [151]:
df

,A,B
0,1,1
0,2,1
0,3,1
1,NaN,1
2,3,1
2,4,1


## 2. Create a heirarchy of levels 

In [152]:
org = pd.read_csv(org_data_path)

In [153]:
org

,dept,parent
0,x,ABCD123
1,ABCD123,ABCD12
2,ABCD12,ABCD1
3,ABCD1,ABCD
4,ABCD,ABC
5,ABC,AB
6,AB,A
7,A,Z


In [154]:
org_temp = org.copy()

In [131]:
#org_temp.rename(columns={'dept':'1', 'parent':'2'},inplace=True)

In [155]:
org_temp

,dept,parent
0,x,ABCD123
1,ABCD123,ABCD12
2,ABCD12,ABCD1
3,ABCD1,ABCD
4,ABCD,ABC
5,ABC,AB
6,AB,A
7,A,Z


In [170]:
org_new = org.copy()
org_new = org_new.rename(columns={'dept': '1', 'parent':'2'})
org_new

,1,2
0,x,ABCD123
1,ABCD123,ABCD12
2,ABCD12,ABCD1
3,ABCD1,ABCD
4,ABCD,ABC
5,ABC,AB
6,AB,A
7,A,Z


In [171]:
for col in range(2,20):
    org_initial = org_temp.rename(columns={'dept':str(col)})
    org_new = org_new.merge(org_initial,how="left")
    org_new = org_new.rename(columns={'parent': str(col+1)})

    org_null_values = org_new[org_new[str(col)].notnull()]
    if org_null_values.shape[0] == 0: 
        org_new = org_new.drop(str(col), axis='columns')
        org_new = org_new.drop(str(col+1), axis='columns')
        break
org_new    

,1,2,3,4,5,6,7,8,9
0,x,ABCD123,ABCD12,ABCD1,ABCD,ABC,AB,A,Z
1,ABCD123,ABCD12,ABCD1,ABCD,ABC,AB,A,Z,NaN
2,ABCD12,ABCD1,ABCD,ABC,AB,A,Z,NaN,NaN
3,ABCD1,ABCD,ABC,AB,A,Z,NaN,NaN,NaN
4,ABCD,ABC,AB,A,Z,NaN,NaN,NaN,NaN
5,ABC,AB,A,Z,NaN,NaN,NaN,NaN,NaN
6,AB,A,Z,NaN,NaN,NaN,NaN,NaN,NaN
7,A,Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
org_new.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='object')

In [172]:
cols = org_new.columns
def create_map(row):
    return [{col:row[col]} for _,col in enumerate(cols)]

#orders['obj'] = orders.apply(create_map, axis=1)

org_new['kvs']= org_new.apply(create_map, axis=1)
org_new

,1,2,3,4,5,6,7,8,9,kvs
0,x,ABCD123,ABCD12,ABCD1,ABCD,ABC,AB,A,Z,"[{'1': 'x'}, {'2': 'ABCD123'}, {'3': 'ABCD12'}..."
1,ABCD123,ABCD12,ABCD1,ABCD,ABC,AB,A,Z,NaN,"[{'1': 'ABCD123'}, {'2': 'ABCD12'}, {'3': 'ABC..."
2,ABCD12,ABCD1,ABCD,ABC,AB,A,Z,NaN,NaN,"[{'1': 'ABCD12'}, {'2': 'ABCD1'}, {'3': 'ABCD'..."
3,ABCD1,ABCD,ABC,AB,A,Z,NaN,NaN,NaN,"[{'1': 'ABCD1'}, {'2': 'ABCD'}, {'3': 'ABC'}, ..."
4,ABCD,ABC,AB,A,Z,NaN,NaN,NaN,NaN,"[{'1': 'ABCD'}, {'2': 'ABC'}, {'3': 'AB'}, {'4..."
5,ABC,AB,A,Z,NaN,NaN,NaN,NaN,NaN,"[{'1': 'ABC'}, {'2': 'AB'}, {'3': 'A'}, {'4': ..."
6,AB,A,Z,NaN,NaN,NaN,NaN,NaN,NaN,"[{'1': 'AB'}, {'2': 'A'}, {'3': 'Z'}, {'4': na..."
7,A,Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'1': 'A'}, {'2': 'Z'}, {'3': nan}, {'4': nan..."


In [173]:
org_new = org_new.loc[ : , ['1','kvs']]
org_bk = org_new.copy()
org_new

,1,kvs
0,x,"[{'1': 'x'}, {'2': 'ABCD123'}, {'3': 'ABCD12'}..."
1,ABCD123,"[{'1': 'ABCD123'}, {'2': 'ABCD12'}, {'3': 'ABC..."
2,ABCD12,"[{'1': 'ABCD12'}, {'2': 'ABCD1'}, {'3': 'ABCD'..."
3,ABCD1,"[{'1': 'ABCD1'}, {'2': 'ABCD'}, {'3': 'ABC'}, ..."
4,ABCD,"[{'1': 'ABCD'}, {'2': 'ABC'}, {'3': 'AB'}, {'4..."
5,ABC,"[{'1': 'ABC'}, {'2': 'AB'}, {'3': 'A'}, {'4': ..."
6,AB,"[{'1': 'AB'}, {'2': 'A'}, {'3': 'Z'}, {'4': na..."
7,A,"[{'1': 'A'}, {'2': 'Z'}, {'3': nan}, {'4': nan..."


In [174]:
org_new = org_new.explode('kvs')

In [177]:
org_new.shape

(72, 2)

https://stackoverflow.com/questions/23586510/return-multiple-columns-from-pandas-apply

In [178]:
import ast

def extract(s):
    depth = next(iter(s['kvs']))
    s['depth'] = depth
    s['parent'] = s['kvs'][depth]
    return s
org_new = org_new.apply(extract,axis=1)
org_new

,1,kvs,depth,parent
0,x,{'1': 'x'},1,x
0,x,{'2': 'ABCD123'},2,ABCD123
0,x,{'3': 'ABCD12'},3,ABCD12
0,x,{'4': 'ABCD1'},4,ABCD1
0,x,{'5': 'ABCD'},5,ABCD
0,x,{'6': 'ABC'},6,ABC
0,x,{'7': 'AB'},7,AB
0,x,{'8': 'A'},8,A
0,x,{'9': 'Z'},9,Z
1,ABCD123,{'1': 'ABCD123'},1,ABCD123


In [184]:
org_new.reset_index(inplace=True)

In [187]:
#df = df.drop(df[df.score < 50].index)
org_new = org_new.drop(org_new[org_new['parent'].isnull()].index)
org_new.drop(columns = ['kvs'], inplace=True)
#org_new[org_new['parent'].isnull()].index

In [179]:
org_new.shape

(72, 4)

In [188]:
org_new

,index,1,depth,parent
0,0,x,1,x
1,0,x,2,ABCD123
2,0,x,3,ABCD12
3,0,x,4,ABCD1
4,0,x,5,ABCD
5,0,x,6,ABC
6,0,x,7,AB
7,0,x,8,A
8,0,x,9,Z
9,1,ABCD123,1,ABCD123


In [144]:
org_bk2 = org_bk.copy()


,1,kvs
0,x,{'1': 'x'}
0,x,{'2': 'ABCD123'}
0,x,{'3': 'ABCD12'}
0,x,{'4': 'ABCD1'}
0,x,{'5': 'ABCD'}
0,x,{'6': 'ABC'}
0,x,{'7': 'AB'}
0,x,{'8': 'A'}
0,x,{'9': 'Z'}
1,ABCD123,{'1': 'ABCD123'}
